In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:08<00:00,  3.73it/s]


eval images shape: (30, 1041, 1302) contain 10076 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 115.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 434.494 || Time Upd.: 412.9 ms  || BatchNr.: 100
 || Cost: 217.894 || Time Upd.: 435.7 ms  || BatchNr.: 200
 || Cost: 184.021 || Time Upd.: 397.6 ms  || BatchNr.: 300
 || Cost: 206.561 || Time Upd.: 391.4 ms  || BatchNr.: 400
 || Cost: 120.465 || Time Upd.: 372.3 ms  || BatchNr.: 500
 || Cost: 109.392 || Time Upd.: 376.1 ms  || BatchNr.: 600
 || Cost: 98.311 || Time Upd.: 373.8 ms  || BatchNr.: 700
 || Cost: 88.416 || Time Upd.: 376.2 ms  || BatchNr.: 800
 || Cost: 70.778 || Time Upd.: 374.6 ms  || BatchNr.: 900
 || Cost: 69.415 || Time Upd.: 374.9 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:6683
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 6683
JoR: 1.010 || Eff_3d: 35.395 || Jaccard: 0.658 || Factor: 359.388 || RMSE_lat: 65.163 || RMSE_ax: 87.717 ||

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7510
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7510
JoR: 1.164 || Eff_3d: 43.986 || Jaccard: 0.719 || Factor: 369.023 || RMSE_lat: 61.753 || RMSE_ax: 68.116 || RMSE_x: 46.501 || RMSE_y: 40.633 || Cost: 47.265 || Recall: 0.730 || Precision: 0.979 || BatchNr.: 2300 || Cost: 47.265 || Time Upd.: 375.7 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 43.986 || 61.753 || 68.116 || 2300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:4159
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 4159
JoR: 0.818 || Eff_3d: 28.082 || Jaccard: 0.413 || Factor: 249.545 || RMSE_lat: 50.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:5602
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 5602
JoR: 1.026 || Eff_3d: 38.922 || Jaccard: 0.554 || Factor: 283.247 || RMSE_lat: 54.020 || RMSE_ax: 54.125 || RMSE_x: 39.368 || RMSE_y: 36.992 || Cost: 41.839 || Recall: 0.555 || Precision: 0.998 || BatchNr.: 3600 || Cost: 41.839 || Time Upd.: 378.3 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 38.922 || 54.020 || 54.125 || 3600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:6362
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 6362
JoR: 1.123 || Eff_3d: 43.937 || Jaccard: 0.630 || Factor: 323.432 || RMSE_lat: 56.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7170
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7170
JoR: 1.232 || Eff_3d: 49.389 || Jaccard: 0.708 || Factor: 326.901 || RMSE_lat: 57.458 || RMSE_ax: 44.710 || RMSE_x: 41.903 || RMSE_y: 39.313 || Cost: 39.788 || Recall: 0.710 || Precision: 0.997 || BatchNr.: 4900 || Cost: 39.788 || Time Upd.: 372.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 49.389 || 57.458 || 44.710 || 4900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:6931
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 6931
JoR: 1.245 || Eff_3d: 49.537 || Jaccard: 0.687 || Factor: 317.192 || RMSE_lat: 55.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:6986
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 6986
JoR: 1.268 || Eff_3d: 50.400 || Jaccard: 0.692 || Factor: 315.628 || RMSE_lat: 54.528 || RMSE_ax: 39.275 || RMSE_x: 39.051 || RMSE_y: 38.057 || Cost: 38.204 || Recall: 0.692 || Precision: 0.999 || BatchNr.: 6200 || Cost: 38.204 || Time Upd.: 376.2 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 50.400 || 54.528 || 39.275 || 6200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7005
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7005
JoR: 1.271 || Eff_3d: 50.492 || Jaccard: 0.695 || Factor: 310.315 || RMSE_lat: 54.

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7321
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7321
JoR: 1.344 || Eff_3d: 53.029 || Jaccard: 0.726 || Factor: 317.783 || RMSE_lat: 54.005 || RMSE_ax: 38.096 || RMSE_x: 38.540 || RMSE_y: 37.831 || Cost: 34.122 || Recall: 0.726 || Precision: 0.999 || BatchNr.: 14000 || Cost: 34.122 || Time Upd.: 373.4 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.029 || 54.005 || 38.096 || 14000


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7278
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7278
JoR: 1.338 || Eff_3d: 52.801 || Jaccard: 0.721 || Factor: 333.463 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7390
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7390
JoR: 1.343 || Eff_3d: 52.808 || Jaccard: 0.733 || Factor: 331.759 || RMSE_lat: 54.558 || RMSE_ax: 40.759 || RMSE_x: 38.720 || RMSE_y: 38.437 || Cost: 33.587 || Recall: 0.733 || Precision: 0.999 || BatchNr.: 15300 || Cost: 33.587 || Time Upd.: 369.6 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 52.808 || 54.558 || 40.759 || 15300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7210
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7210
JoR: 1.302 || Eff_3d: 52.023 || Jaccard: 0.715 || Factor: 328.619 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7474
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7474
JoR: 1.358 || Eff_3d: 53.709 || Jaccard: 0.741 || Factor: 329.001 || RMSE_lat: 54.559 || RMSE_ax: 38.110 || RMSE_x: 38.582 || RMSE_y: 38.576 || Cost: 33.037 || Recall: 0.741 || Precision: 0.999 || BatchNr.: 16600 || Cost: 33.037 || Time Upd.: 371.4 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.709 || 54.559 || 38.110 || 16600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7682
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7682
JoR: 1.341 || Eff_3d: 52.255 || Jaccard: 0.761 || Factor: 329.935 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7816
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7816
JoR: 1.357 || Eff_3d: 53.496 || Jaccard: 0.775 || Factor: 333.153 || RMSE_lat: 57.095 || RMSE_ax: 44.485 || RMSE_x: 41.186 || RMSE_y: 39.542 || Cost: 32.958 || Recall: 0.775 || Precision: 0.999 || BatchNr.: 17900 || Cost: 32.958 || Time Upd.: 371.4 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.496 || 57.095 || 44.485 || 17900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7459
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7459
JoR: 1.354 || Eff_3d: 53.723 || Jaccard: 0.740 || Factor: 328.877 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7431
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7431
JoR: 1.342 || Eff_3d: 53.317 || Jaccard: 0.737 || Factor: 319.992 || RMSE_lat: 54.923 || RMSE_ax: 38.080 || RMSE_x: 38.576 || RMSE_y: 39.095 || Cost: 33.169 || Recall: 0.737 || Precision: 1.000 || BatchNr.: 19200 || Cost: 33.169 || Time Upd.: 373.2 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 53.317 || 54.923 || 38.080 || 19200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10076, preds:7423
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10076 ,preds: 7423
JoR: 1.346 || Eff_3d: 53.285 || Jaccard: 0.736 || Factor: 325.992 || RMSE_lat: 5